#### Sociology 128D: Mining Culture Through Text Data: Introduction to Social Data Science – Summer '22

# Notebook 9: Bias and Stereotyping in Pretrained Word Embeddings

In this notebook, we will use pretrained word embeddings to develop intuitions about how word embedding models can be used to answer social research questions. We start with a brief introduction to word embeddings in general and a demonstration of solving analogies using word vectors.

Next, we draw inspiration from ["The Geometry of Culture" (Kozlowski, Taddy, & Evans, 2019)](https://journals.sagepub.com/doi/full/10.1177/0003122419877135) and explore how word embedding models can be used to study cultural associations, for example classed, racialized, and gendered perceptions of music.

## What are Word Embeddings?

Word *vectors*, generally speaking, are vectors (lists of numbers) that represent the positions of distinct words in a shared embedding space. They are similar to the tf-idf-weighted document vectors we used in [Notebook 5](https://soc128d.github.io/notebooks/) in that the unit of analysis is represented by a vector of numbers and these numbers reflect information about the co-occurrence of words in a corpus. In the case of word vectors, however, the unit of analysis is the individual word, not a document.

In general, word embeddings capture patterns of similar usage of words. Words used in similar contexts will be closer together in the embedding space. Synonyms are the extreme case: if you replace one synonym with another in a sentence (e.g., replace "car" with "automobile"), the meaning of the sentence may not change a great deal. Both "car" and "automobile" could reasonably be used in that sentence, and both versions of that sentence (or sentences like it) could occur frequently in a corpus. The other words in the sentence are <tt>context words</tt> (and, in fact, every word is a context word for every other word in a given sentence). Words used frequently in the presence of the same context words are assumed to be similar, and we want word vectors that reflect this.

In Notebooks 3 and 5, we used a document X term (word) matrix to represent documents as vectors. To represent individual *words* as vectors, we could use that approach, but word vectors based on a document-term matrix would reflect the extent to which words are used in the same documents. If we want to represent the words as vectors based on how similarly they are used to other individual words, we want something like a word X _context word_ matrix representing the number of times words co-occur (or are "context words" for one another) in, say, individual sentences.

The issue with a word X *context word* matrix is that such a matrix is often very sparse: it might be very large if we have a large vocabulary, and many words may never directly co-occur. Other approaches lead to "dense vectors" in which columns do not correspond to individual context words. Despite having a vocabulary of size V (which may be in the thousands or even millions), we often learn representations of words using only 50-300 dimensions. These lower-dimensional representations are typically what we refer to when we talk about **word embeddings**.

We can use measures like cosine similarity to quantify the distance in the embedding space, and this reflects similarity in how words are used. We often make the further leap of assuming that this measures similarity in what words _mean_. [Sahlgren (2008, p. 1)](https://www.diva-portal.org/smash/get/diva2:1041938/FULLTEXT01.pdf) points to numerous formulations of this claim:

> Distributional approaches to meaning acquisition utilize distributional properties of linguistic entities as the building blocks of semantics. In doing so, they rely fundamentally on a set of assumptions about the nature of language and meaning referred to as the distributional hypothesis. This hypothesis is often stated in terms like “words which are similar in meaning occur in similar contexts” (Rubenstein & Goodenough, 1965); “words with similar meanings will occur with similar neighbors if enough text material is available” (Sch ̈utze & Pedersen, 1995); “a representation that captures much of how words are used in natural context will capture much of what we mean by meaning” (Landauer & Dumais, 1997); and “words that occur in the same contexts tend to have similar meanings” (Pantel, 2005), just to quote a few representative examples. The general idea behind the distributional hypothesis seems clear enough: there is a correlation between distributional similarity and meaning similarity, which allows us to utilize the former in order to estimate the latter.

There are a number of approaches to training word embeddings, but one of the most common–as we've seen in various readings–is word2vec. **You can find an excellent illustrated guide to word2vec [here](https://jalammar.github.io/illustrated-word2vec/).** In brief, the word2vec algorithm involves either iteratively predicting a target word based on the context words in a sentence or, alternatively, predicting the context words (and *not* predicting certain wrong words, or "negative samples") from the target word. With each iteration, representations of words are moved closer together if they are used in similar ways. Another common approach to training word embeddings is called [GloVe](https://nlp.stanford.edu/projects/glove/), and this is based more directly on a word X word co-occurrence matrix. More sophisticated approaches have been developed, and a lot of attention has been given to word embeddings from language models like BERT, but we are going to stick with word2vec and GloVe for this notebook.

[Jurafsky and Martin (2021)](https://web.stanford.edu/~jurafsky/slp3/) have an excellent chapter on vector semantics and embeddings, which you can find [here](https://web.stanford.edu/~jurafsky/slp3/6.pdf).

## Stereotyping and Bias in Word Embeddings

Word embeddings are attempts to represent words as vectors such that words with similar meanings will have similar vectors. Word embeddings are learned based on how words are used in a corpus and, as a result, may be influenced by bias inherent to a given corpus. In general, however, stereotyping and bias are a part of natural language. From an engineering standpoint, we want to remove bias from word embeddings (or **debias** them) so that our applications do not perpetuate stereotypes or treat people in unfair ways. From a social science standpoint, however, we may want to explore stereotyping or bias directly.

The classic example of a paper on detecting bias and debiasing word embeddings is [Bolukbasi et al. (2016)](https://proceedings.neurips.cc/paper/2016/hash/a486cd07e4ac3d270571622f4f316ec5-Abstract.html). The title of the paper–"Man is to Computer Programmer as Woman is to Homemaker?"–says it all. As we'll see, you can quite effectively complete analogies using word embeddings, but completing those analogies may rely on assumptions about topics like gender. [Garg et al. (2018)](https://www.pnas.org/doi/full/10.1073/pnas.1720347115) offer another classic example of using word embeddings to study stereotyping and bias. 

## Setup

You'll need a version of the `gensim` library. You can install `gensim` from the Anaconda interpreter using the following command:

```
conda install -c anaconda gensim
```

If you are not using Anaconda, you can use `pip` to install `gensim` from the command line:

```
pip3 install gensim
```

This notebook has been tested with versions 3.8.3, 4.0.1, and 4.1.2. In the shift from version 3 to version 4, [a few notable changes were made](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4). For this notebook, the only difference that should matter is that word embedding models had a <tt>.vocab</tt> attribute in version 3, and this has been replaced with <tt>.key_to_index</tt> in version 4. There are a couple of lines of code in this notebook that use one of these attributes; they are flagged for you, and you can simply make sure to comment out the version you don't need and uncomment the version you do. If you're using version 4, use <tt>.key_to_index</tt>. If you're using an older version, use the lines of code with <tt>.vocab</tt>.

Later in the notebook, you'll explore latent cultural dimensions measured as in ["The Geometry of Culture"](https://journals.sagepub.com/doi/full/10.1177/0003122419877135). You will create these using pairs of antonyms provided by the authors. The notebook links to these lists of words, and you can access them using either a function that will load them from your computer (if you've downloaded them) or a different function that will read them directly from the web. If you plan to use the word pairs on your own (e.g., for your project) or plan to run the notebook multiple times, it might be best to download the lists and use the function provided to load them. Otherwise, getting them directly from the web should be fine.

In [ ]:
import gensim.downloader
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests

from sklearn.metrics.pairwise import cosine_similarity

sns.set_theme(style="darkgrid")

`sklearn`'s `cosine_similarity()` method expects vectors to be in a certain shape. The <tt>cosine_reshape()</tt> function below is a helper function to handle that for us.

In [ ]:
def cosine_reshape(vec1, vec2):
    """
    Reshape word vectors and return cosine
    similarity
    """
    if vec1.shape[0] > 1:
        vec1 = vec1.reshape(1, -1)
    if vec2.shape[0] > 1:
        vec2 = vec2.reshape(1, -1)
    assert vec1.shape == vec2.shape
    return cosine_similarity(vec1, vec2)[0][0]

Let's load our word embedding model. The notebook defaults to [<tt>word2vec-google-news-300</tt>](https://github.com/RaRe-Technologies/gensim-data), which has a vocabulary of three million words, has three hundred dimensions, and was trained on a Google New corpus. You can try other pretrained embeddings if you prefer. `gensim` also vectors trained using algorithms like [GloVe](https://nlp.stanford.edu/projects/glove/), with versions trained on different types of data and with different numbers of dimensions. Typically, smaller models (i.e., with smaller vocabularies or fewer dimensions) are faster to work with, but may not represent the meaning of the words as well.

<b>Note: If you are using a particular model for the first time, the following cell will try to print a progress bar for the download, and you may get repeated warnings. It should continue to run despite these warnings. Once it has downloaded, you can run the cell again, or even comment it out, to hide the output.<br>

In [ ]:
# word_vectors = gensim.downloader.load("glove-wiki-gigaword-300") # if you'd prefer to try GloVe vectors instead!
word_vectors = gensim.downloader.load("word2vec-google-news-300")

In [ ]:
type(word_vectors)

#### If you have `gensim` 4.0 and above, you'll need to comment out the lines with references to the <tt>.vocab</tt> attribute and uncomment the lines that reference to the <tt>.key_to_index</tt> attribute.

In [ ]:
gensim.__version__

In [ ]:
type(word_vectors.key_to_index)
# type(word_vectors.vocab)

In [ ]:
print(f"There are {len(word_vectors.key_to_index):,} words in the vocabulary.")
# print(f"There are {len(word_vectors.vocab):,} words in the vocabulary.")

Let's take a look at an example of a word embedding.

In [ ]:
word_vectors["sociology"]

The embedding for sociology indicates its position in a 300-dimensional space, but there is a vocabulary of three million words. The dimensions do not correspond to context words, and do not necessarily correspond to anything interpretable on their own.

In [ ]:
len(word_vectors["sociology"])

## Similarity and Analogies

`gensim` makes it easy to compare word embeddings. the `.similarity()` method takes two words as arguments and provides the cosine similarity of the corresponding word embeddings. This is the same measure of similarity we used for comparing document vectors.

`gensim`'s `.most_similar()` method takes one word as an argument and provides the <tt>topn</tt> most similar words in the model. However, the `.most_similar()` method also allows you to add and subtract word vectors.

In [ ]:
word_vectors.similarity("sociology", "anthropology")

In [ ]:
word_vectors.most_similar("sociology", topn=10)

The typical example of analogies with word vectors is finding a feminine counterpart of "king" by addition and subtraction of other vectors. Starting with the vector for king, we can subtract the vector for "man" and add the vector for "woman."

The analogy takes the following form:
```
 man:king::woman:___
```
Using the `.most_similar()` method, we provide a `list` called <tt>positive</tt> that includes "king" and "woman" and a `list` called <tt>negative</tt> including only "man." You can think of this as adding the vector for "woman" to the vector for "king" while subtracting the vector for "man."

In [ ]:
word_vectors.most_similar(positive=["king", "woman"], negative=["man"], topn=10)

Word vectors can also seemingly pick up aspects of grammar:

```
wide:wider::light:___
read:reading::listen:___
write:wrote::listen:___
```

In [ ]:
word_vectors.most_similar(positive=["wider", "light"], negative=["wide"], topn=1)

In [ ]:
word_vectors.most_similar(positive=["reading", "listen"], negative=["read"], topn=1)

In [ ]:
word_vectors.most_similar(positive=["wrote", "listen"], negative=["write"], topn=1)

What about when we subtract the vector for "society" from the vector for "sociology" and add the vector for "cell"?

```
society:sociology::cell:___
```

In [ ]:
word_vectors.most_similar(positive=["sociology", "cell"], negative=["society"], topn=10)

## Measuring "Cultural Dimensions"

#### If you would prefer to download and save the lists of word pairs used by Kozlowski et al. (2019):

Affluence pairs: https://raw.githubusercontent.com/KnowledgeLab/GeometryofCulture/master/code/word_pairs/affluence_pairs.csv

Gender pairs: https://raw.githubusercontent.com/KnowledgeLab/GeometryofCulture/master/code/word_pairs/gender_pairs.csv

Race pairs: https://raw.githubusercontent.com/KnowledgeLab/GeometryofCulture/master/code/word_pairs/race_pairs.csv

In [ ]:
def read_pairs_from_file(f: str) -> list:
    """
    Read in word pairs from `f` as list
    """
    
    pairs = open(f, "r").read().strip().split("\n")
    pairs = [pair.split(",") for pair in pairs]
    return [(pair[0], pair[1]) for pair in pairs]

In [ ]:
aff_f = "affluence_pairs.csv"
gender_f = "gender_pairs.csv"
race_f = "race_pairs.csv"

aff_pairs = read_pairs_from_file(aff_f)
gender_pairs = read_pairs_from_file(gender_f)
race_pairs = read_pairs_from_file(race_f)

#### If you would prefer to get the lists of word pairs used by Kozlowski et al. (2019) using the `requests` library:

In [ ]:
def read_pairs_from_url(url: str) -> list:
    """
    Get word pairs using requests library
    """
    
    pairs = requests.get(url)
    pairs = pairs.text.strip().split("\n")
    pairs = [pair.replace("\r", "").split(",") for pair in pairs]
    return [(pair[0], pair[1]) for pair in pairs]

In [ ]:
aff_url = "https://raw.githubusercontent.com/KnowledgeLab/GeometryofCulture/master/code/word_pairs/affluence_pairs.csv"
gender_url = "https://raw.githubusercontent.com/KnowledgeLab/GeometryofCulture/master/code/word_pairs/gender_pairs.csv"
race_url = "https://raw.githubusercontent.com/KnowledgeLab/GeometryofCulture/master/code/word_pairs/race_pairs.csv"

aff_pairs = read_pairs_from_url(aff_url)
gender_pairs = read_pairs_from_url(gender_url)
race_pairs = read_pairs_from_url(race_url)

In [ ]:
aff_pairs

In [ ]:
gender_pairs

In [ ]:
race_pairs

The function below is a recreation of the <tt>make_dim()</tt> function in the original `R` code provided by Kozlowski et al. [here](https://github.com/KnowledgeLab/GeometryofCulture/blob/master/code/build_cultural_dimensions.R). Like other research on bias in word embeddings, this work presupposes cultural associations that can be represented as binaries or continua.

To identify an "affluence" dimension, this approach uses the differences of vectors for antonyms like "rich" and "poor" (e.g., "rich" minus "poor").

To identify "gender" and "race" dimensions, this approach makes the assumption that _culturally_ there are axes where the idea of gender or ethnic "antonyms" makes sense. If there is gender bias such that some things are stereotyped as being more associated with men than women (or vice versa), this approach may be able to detect it.

In [ ]:
def make_dim(word_pairs: list) -> np.ndarray:
    """
    Calculate a "cultural dimension" as in
    https://github.com/KnowledgeLab/GeometryofCulture/blob/master/code/build_cultural_dimensions.R
    """
    
    diffs = []
    
    for pair in word_pairs:
        
        word1, word2 = pair
        
        if word1 in word_vectors and word2 in word_vectors:

            vec1 = word_vectors[pair[0]]
            vec1 = vec1/np.linalg.norm(vec1) # norm the first word vector

            vec2 = word_vectors[pair[1]]
            vec2 = vec2/np.linalg.norm(vec2) # norm the second second vector

            diff = vec1 - vec2
            diff = diff/np.linalg.norm(diff) # norm the difference between the vectors
            diffs.append(diff)               # append the difference to the list
            
        else:
            print(f"Missing: {pair}") # missing words from affluence pairs
        
    dim = np.mean(diffs, axis=0) # average the vectors appended to the list
    dim = dim/np.linalg.norm(dim) # norm the average
    
    return dim

Now we can calculate the cultural dimensions!

In [ ]:
aff_dim = make_dim(aff_pairs)
gender_dim = make_dim(gender_pairs)
race_dim = make_dim(race_pairs)

Each is a 300-dimensional vector just like the word vectors, but it doesn't correspond to a single word.

In [ ]:
type(aff_dim)

In [ ]:
aff_dim.shape

#### Locating musical genres along these dimensions

Now we can create a plot like Figure 3 in the original paper (p. 921). We won't overlay a plot based on survey data.

In [ ]:
genres = ["jazz", "rap", "opera", "punk", "techno", "hiphop", "bluegrass"]

In [ ]:
aff_proj_dict = {}
gender_proj_dict = {}
race_proj_dict = {}

for genre in genres:
    cos_aff = cosine_reshape(word_vectors[genre], aff_dim)
    aff_proj_dict[genre] = cos_aff
    
    cos_gender = cosine_reshape(word_vectors[genre], gender_dim)
    gender_proj_dict[genre] = cos_gender
    
    cos_race = cosine_reshape(word_vectors[genre], race_dim)
    race_proj_dict[genre] = cos_race

The results are dictionaries assigning a position to each musical genre along a particular dimension.

For the affluence dimension, higher values indicate the words are closer to the more affluent side.

For the gender dimension, higher values indicate words are more associated with men or masculinity (relative to women or femininity).

For the race dimension, higher values indicate words are potentially more stereotyped as Black (relative to white).

In [ ]:
aff_proj_dict

In [ ]:
gender_proj_dict

In [ ]:
race_proj_dict

In [ ]:
plt.figure(figsize=(14, 8))

for genre in genres:
    plt.text(x=race_proj_dict[genre], y=aff_proj_dict[genre], s=genre)

# These are set to work with the word2vec embeddings. You may need to change the limits of the axes if you use GloVe.
plt.xlim(-0.05,0.3)
plt.ylim(-0.10, 0.15)

plt.xlabel("Race Projection")
plt.ylabel("Class Projection")
plt.title("Class and Racial Associations of Musical Genres")
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))

for genre in genres:
    plt.text(x=race_proj_dict[genre], y=gender_proj_dict[genre], s=genre)
    
plt.xlim(-0.05,0.3)
plt.ylim(-0.10, 0.15)

plt.xlabel("Race Projection")
plt.ylabel("Gender Projection")
plt.title("Gender and Racial Associations of Musical Genres")
plt.show()

Importantly, Kozlowski et al. (2019) compare their results to survey data and find similar associations between musical genres and these projections.

## Exercises

<div class="alert alert-warning">
    <b>Exercise 1</b><br><br>
    For this exercise, you will identify another domain of social life that may exhibit the kind of bias we see above.<br><br>
    1.1 Create a list of things associated with the domain you have identified, just like the list of musical genres above (<tt>genres</tt>). Only include words that are in the vocabulary of the word embedding model you are using.
</div>

In [ ]:
# YOUR CODE HERE

You can use the `filter()` method and a lambda function to get rid of any words that aren't in the word embedding model. Assuming your word embedding model is named <tt>word_vectors</tt> (as above) and your list is called <tt>list_of_things</tt> (which is optional), the following line would keep only the words that are in the word embedding model's vocabulary.

```python
list_of_things = list(filter(lambda x: x in word_vectors, list_of_things))
```

<div class="alert alert-warning">
    1.2 What domain did you choose? Why? What associations do you expect to see with the cultural dimensions we've measured?
</div>

_Your answer here_

<div class="alert alert-warning">
    1.3 Replace <tt>LIST_OF_THINGS</tt> with the name of your list and run the cell below.
</div>

In [ ]:
for element in LIST_OF_THINGS: # replace LIST_OF_THINGS
    cos_aff = cosine_reshape(word_vectors[element], aff_dim)
    aff_proj_dict[element] = cos_aff
    
    cos_gender = cosine_reshape(word_vectors[element], gender_dim)
    gender_proj_dict[element] = cos_gender
    
    cos_race = cosine_reshape(word_vectors[element], race_dim)
    race_proj_dict[element] = cos_race

<div class="alert alert-warning">
    1.4 Now plot the positions of the words you chose in a two-dimensional space defined by two of the three cultural dimensions we've measured. (The code in the cells with the plots above should only need minor tweaks to plot your words.)
</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">
    1.5 Do the results match the associations you predicted in 1.2?
</div>

_Your answer here_

<div class="alert alert-warning">
    <b>Exercise 2</b><br><br>
    For this exercise, you will identify and measure a new cultural dimension.<br><br>
    2.1 Begin by identifying pairs of words that should be at opposite ends of the dimension. These may be antonyms like "rich" and "poor" or they may be anchors for a cultural dimension related to stereotyping and bias that aren't necessarily antonyms. (For example, "Black" and "White" as ethnicities aren't antonyms, but are useful anchors for identifying a cultural dimension that allows us to measure and visualize bias.) <br><br>
    You can create this list of word pairs as a list of lists or list of tuples inside the notebook, or you might try saving them in a CSV file and opening them with the <tt>read_pairs_from_file()</tt> function defined above. NOTE: The anchors should always be in the same order. For example, in <tt>affluence_pairs.csv</tt>, words associated with the affluent end of the spectrum are always listed first, and the antonyms are listed the second. You can look at the CSVs used above for guidance.
</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">
    2.2 What is your cultural dimension meant to measure? What kinds of stereotypes might you see?
</div>

_Your answer here_

<div class="alert alert-warning">
    2.3 Calculate the cultural dimension using the <tt>make_dim()</tt> function defined above. This should look like the following (but with your word pairs, and a name other than <tt>aff_dim</tt>):
</div>

```python
aff_dim = make_dim(aff_pairs)
```

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">
    2.4 Now using your dimension and one of the others, plot a list of words in the two-dimensional space defined by the dimensions like in 1.4.<br><br>
    - Rename <tt>YOUR_DIM_PROJ_DICT</tt><br>
    - Replace <tt>LIST_OF_THINGS</tt> with the list you want to use (which could be musical genres, the list you used in Exercise 1, or something else)<br>
    - Replace <tt>YOUR_DIM</tt> with the name of the variable to which you've assigned your cultural dimension<br>
    - Replace <tt>YOUR_DIM_PROJ_DICT</tt> in the last line with the name you chose in the first line<br>
    - Create a plot using your dimension and one other dimension
</div>

In [ ]:
YOUR_DIM_PROJ_DICT = {} # rename this

for element in LIST_OF_THINGS: # replace LIST_OF_THINGS
    cos_aff = cosine_reshape(word_vectors[element], aff_dim)
    aff_proj_dict[element] = cos_aff
    
    cos_gender = cosine_reshape(word_vectors[element], gender_dim)
    gender_proj_dict[element] = cos_gender
    
    cos_race = cosine_reshape(word_vectors[element], race_dim)
    race_proj_dict[element] = cos_race
    
    cos_new = cosine_reshape(word_vectors[element], YOUR_DIM) # replace YOUR_DIM with the name of your cultural dim.
    YOUR_DIM_PROJ_DICT[element] = cos_new # replace the name of the dictionary with the name you chose above

In [ ]:
# YOUR PLOT CODE HERE

<div class="alert alert-warning">
    2.5 Do the results match the associations you predicted in 2.2?
</div>

_Your answer here_